In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
anime = pd.read_csv("anime.csv")
reviews = pd.read_csv("rating.csv")


In [3]:
anime.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [4]:
reviews.head()

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


In [5]:
len(reviews)

7813737

In [6]:
len(anime)

12294

In [7]:
#rating = reviews.values

In [8]:
# all_users = {}
# count = len(reviews)
# user = np.array([])
# for i in range(count):
#     user.append(rating[1,0])

# for user in rating.user_id.unique():
#     l = len(rating.user_id == user)
#     temp = {}
#     new = rating[rating.user_id == user]
#     for i in range(l):
#         try:
#             temp[new.loc[i][1]] = new.loc[i][2]
#         except:
#             pass
#     all_users[user] = temp
#     print(all_users)

#Check BROOO
    

In [9]:
count = len(reviews)
preferences = {}

In [10]:
#Numpy array
review_array = reviews.values

In [11]:
for i in range(count):
    user = review_array[i, 0]
    anime = review_array[i, 1]
    rating = review_array[i, 2]
    
    if user not in preferences.keys():
        preferences[user] = {}
    
    preferences[user][anime] = rating

In [12]:
#similarity based on Euclidian
def sim_euclidian(preferences, person1,person2):
    
    similarity = {}
    
    #getting similar anime watched
    
    for anime in preferences[person1]:
        if anime in preferences[person2]:
            similarity[anime] = 1
    
    if len(similarity) == 0:
        return 0
    
    Euclidian_distance = np.sqrt(np.sum([np.power(preferences[person1][anime] - preferences[person2][anime],2) 
                                 for anime in preferences[person1] if anime in preferences[person2]]))
    
    return 1/(1 + Euclidian_distance)

    

In [13]:
#ranking the reviewers
def top_matches(preferences, person, n = 5, similarity = sim_euclidian):
    
    scores = [(similarity(preferences,person,reviewer),reviewer) for reviewer in preferences if reviewer != person]
    
    scores.sort()
    scores.reverse()
    return scores[:n]

In [14]:
top_match = top_matches(preferences,1)
#top_match[0]

In [15]:
top_match

[(1.0, 73472), (1.0, 73467), (1.0, 73418), (1.0, 73411), (1.0, 73407)]

In [21]:
def recommendations(preferences, person, similarity = sim_euclidian):
    total = {}
    simsum = {}
    
    for reviwer in preferences:
        if reviwer == person:
            continue
        sim = similarity(preferences,person,reviwer)
        
        if sim <= 0:
            continue
        for anime in preferences[reviwer]:
            
            if anime not in preferences[person] or preferences[person][anime] == 0:
                total.setdefault(anime,0)
                total[anime] += preferences[reviwer][anime] * sim
                
                simsum.setdefault(anime,0)
                simsum[anime] += sim
                
    ranks = [(total/simsum[anime],anime) for anime,total in total.items()]
    
    ranks.sort()
    ranks.reverse()
    return ranks[:10]

In [22]:
anime = pd.read_csv("anime.csv")
anime_array = anime.values

In [23]:
anime_array[0][0]

32281

In [24]:
top_rec = recommendations(preferences,1)


In [25]:
anime_list = []
for i in top_rec:
    for j in range(len(anime_array)):
        if i[1] == anime_array[j][0]:
            anime_list.append(anime_array[j])
    
#     anime_list.append(anime["name"][anime.anime_id == i[1]])
    

In [26]:
anime_list

[array([33484, 'Shiroi Zou', 'Action, Historical, Kids', 'Movie', '1',
        4.71, 45], dtype=object),
 array([34252, 'STAR BEAT!: Hoshi no Kodou', 'Music', 'Music', '1', 6.39,
        230], dtype=object),
 array([33791, 'Warui no wo Taose!! Salaryman Man', 'Comedy, Parody',
        'ONA', '1', 5.04, 74], dtype=object),
 array([31211, 'Choegang Top Plate', 'Action, Comedy, Kids, Sports', 'TV',
        '26', 7.2, 86], dtype=object),
 array([9820, 'McDull, the Alumni', 'Comedy', 'Movie', '1', 6.38, 83],
       dtype=object),
 array([9819, 'Mak Dau Xiang Dang Dang', 'Comedy', 'Movie', '1', 6.81, 103],
       dtype=object),
 array([9817, 'Mak Dau Goo Si', 'Comedy', 'Movie', '1', 7.02, 136],
       dtype=object),
 array([32422, 'Doukyuusei', 'Kids, Music', 'Music', '1', 5.83, 116],
       dtype=object),
 array([31972, 'Tang Lang Bu Chan', 'Historical', 'Movie', '1', 5.9, 112],
       dtype=object),
 array([30939, 'Shiranpuri', 'Drama, School', 'OVA', '1', 6.18, 155],
       dtype=object)]

In [27]:
top_rec

[(10.000000000000002, 33484),
 (10.0, 34252),
 (10.0, 33791),
 (10.0, 31211),
 (9.541666666666668, 9820),
 (9.541666666666668, 9819),
 (9.541666666666668, 9817),
 (9.206914430351546, 32422),
 (9.0, 31972),
 (9.0, 30939)]